In [1]:
pip install pyobdc

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyobdc (from versions: none)
ERROR: No matching distribution found for pyobdc


In [2]:

pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [32]:
import pandas as pd
from collections import deque
import heapq

class Program:
    def __init__(self, inst_code, brcode, quota, cat, subcat, gender, capacity):
        self.inst_code = inst_code
        self.brcode = brcode
        self.capacity = capacity
        self.gender = gender
        self.quota = quota
        self.cat = cat
        self.subcat = subcat
        self.waitlist = []  # Will store tuples (negative rank, roll_no)

    def add_candidate(self, rk, roll_no):
        # Insert candidate into the heap as a tuple of (-rank, roll_no)
        heapq.heappush(self.waitlist, (-rk, roll_no))

    def remove_least_preferred(self):
        # Remove the candidate with the highest positive rank (least preferred)
        return heapq.heappop(self.waitlist)

class Candidate:
    def __init__(self, roll_no, cat, subcat, gender,  adv_rank, mains_rank):
        self.roll_no = roll_no
        self.gender = gender
        self.cat = cat
        self.subcat = subcat
        self.adv = adv_rank
        self.mains = mains_rank
        self.current_index = 1  # Initialize to 1 assuming choice numbers start from 1

def load_candidates(filename):
    df = pd.read_csv(filename)
    return {row['RollNo']: Candidate(row['RollNo'], row['Cat'], row['SubCat'], row['Gender'], row['Adv_CRL_Rank'], row['AI_Eng_CRL_Rank']) for index, row in df.iterrows()}

def load_programs(filename):
    df = pd.read_csv(filename)
    return {(row['InstCode'], row['Brcode'], row['Quota'], row['Cat'], row['SubCat'], row['Gender']): Program(row['InstCode'], row['Brcode'], row['Quota'], row['Cat'], row['SubCat'], row['Gender'], row['TotalSeat']) for index, row in df.iterrows()}

def load_choices(filename):
    df = pd.read_csv(filename)
    return df

candidates = load_candidates(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Candidate.csv')
programs = load_programs(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Seats.csv')
choices = load_choices(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Choice.csv')



In [18]:
def load_allotment(filename):
    df = pd.read_csv(filename)
    filtered_df = df[(df['Flag'] == 'N') | (df['Flag'] == 'D')]
    
    for index, row in filtered_df.iterrows():
        roll_no = row['RollNO']
        choice_no = row['ChoiceNo']
        candidates[roll_no].current_index = choice_no
        
        inst_code = row['InstCode']
        brcode = row['Brcode']
        quota = row['AllottedQuota']
        cat = row['AllottedCategory'][:2]
        subcat = row['AllottedCategory'][2:]
        gender = row['AllottedGender']
        
        program_key = (inst_code, brcode, quota, cat, subcat, gender)
        program = programs.get(program_key)
        if program:
            rk = candidates[roll_no].adv if inst_code < 200 else candidates[roll_no].mains
            program.add_candidate(rk, roll_no)

    return allotment


allotment = load_allotment(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round1\Round1_Allotment.csv')    


In [33]:
def load_allotment(filename):
    df = pd.read_csv(filename)
    filtered_df = df[(df['Flag'] == 'N') | (df['Flag'] == 'D')]
    
    for index, row in filtered_df.iterrows():
        if row['Withdraw']=='N' and row['RStatus']=='RP':
            roll_no = row['RollNo']
            choice_no = row['AllotedOpt']
            candidates[roll_no].current_index = choice_no
            
            inst_code = row['Instcd']
            brcode = row['Brcd']
            quota = row['AllottedQuota']
            cat = row['AllottedCat'][:2]
            subcat = row['AllottedCat'][2:]
            gender = row['AllottedGender']
            
            program_key = (inst_code, brcode, quota, cat, subcat, gender)
            program = programs.get(program_key)
            if program:
                rk = candidates[roll_no].adv if inst_code < 200 else candidates[roll_no].mains
                program.add_candidate(rk, roll_no)
    return allotment


allotment = load_allotment(r'C:\Users\student\Desktop\JOSAA\JOSAA Prev\Test1\Round2\Candidate.csv')    

In [34]:
print("\nFinal Waitlists:")
for program_key, program in programs.items():
    if(len(program.waitlist)>0):
        print(f"Program {program_key} Cutoff : {-program.waitlist[0][0]} waitlist: {program.waitlist} with {program.capacity-len(program.waitlist)} remaining seats")



Final Waitlists:
Program (101, '4109', 'AI', 'BC', 'NO', 'B') Cutoff : 18328.0 waitlist: [(-18328.0, 230310334386), (-18013.0, 230310437072), (-18230.0, 230310384837), (-17725.0, 230310270254), (-17936.0, 230310333498), (-18121.0, 230310143253), (-18226.0, 230310605377), (-16154.0, 230310447036), (-17090.0, 230310710337), (-17083.0, 230310448822), (-17797.0, 230310379188), (-16546.0, 230310181721), (-17241.0, 230310008065), (-17140.0, 230310552460)] with 0 remaining seats
Program (101, '4109', 'AI', 'BC', 'NO', 'F') Cutoff : nan waitlist: [(nan, 230310119905), (-22671.0, 230310556109)] with 2 remaining seats
Program (101, '4109', 'AI', 'EW', 'NO', 'B') Cutoff : nan waitlist: [(nan, 230310863248), (-13694.0, 230310173345), (-13668.0, 230310176698), (-13240.0, 230310051651)] with 1 remaining seats
Program (101, '4109', 'AI', 'EW', 'NO', 'F') Cutoff : 19893.0 waitlist: [(-19893.0, 230310565164)] with 0 remaining seats
Program (101, '4109', 'AI', 'OP', 'NO', 'B') Cutoff : 12911.0 waitlist